In [ ]:
import pandas as pd
import numpy as np
import pickle
import urllib.request
import dateutil.parser
import dateutil.rrule
import dateutil.tz
import datetime
import warnings
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

## Summary

These tables show a selection of the monitoring points on the road network across Tyne and Wear. The points have been selected because they indicate long distance travel or commuting behaviour. The flows are obtained using a network of ANPR cameras used by Tyne and Wear UTMC, normally used to monitor journey times and invoke traffic management strategies. No actual number plates are used in the creation of these statistics, only the number of plates recognised.

For each day, the following percentages are given with respect to the average vehicles per hour:
 * Change since the day before
 * Change since the same weekday last week
 * Change compared to the median for the same weekday calculated over the last year
 
Statistics are provided for the morning peak, afternoon peak, inter-peak period (essentially daytime outside of peak hours), and night. As vehicle traffic is unlikely to be evenly distributed across these periods, the statistics for the current period will be volatile while data is still coming in.

In [ ]:
# Used across most of the plots for people flows
tzLocal = dateutil.tz.gettz('Europe/London')
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time()).replace(tzinfo=tzLocal)

trafficCountInterval = 900
pdTrafficCount = pickle.load(open('../cache/recent-traffic-volumes-pd.pkl', 'rb'))

# TODO: Make this reflect the last entry in the frame, not the time now
print('Last data obtained %s' 
    % (np.max(pdTrafficCount.index).strftime('%d %B %Y %H:%M')))

In [ ]:
# Ignore non-numeric columns in the dataframe
plottableTypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
perMinuteFactor = (trafficCountInterval / 60)

daysToInclude = 15
daysToSkip = ['17/03/2020', '18/03/2020'] # Data collection wasn't fully in place

In [ ]:
summaryPoints = {
    'Traffic volumes from A1 towards Newcastle upon Tyne': {
        'J66 Angel (Northbound)': 'CAJT_GHA167_NB4_DR3.start.northbound',
        'J69 Redheugh (Northbound)': 'CAJT_GHA184_DWR1_RB.start.eastbound',
        # 'J75 Denton (Eastbound)': 'CAJT_NCA186_WR4_WR3.start', # Denton data seems dodgy to me...
        'J76 Blakelaw (Eastbound)': 'CAJT_NCB6324_SR2_SR1.start.eastbound'
    },
    'Traffic volumes at A1 leaving Newcastle upon Tyne': {
        'J66 Angel (Southbound)': 'CAJT_GHA167_DR3_NB4.end.southbound',
        'J69 Redheugh (Soutbound)': 'CAJT_GHA184_RB_A1.end.westbound',
        # 'J75 Denton (Westbound)': 'CAJT_NCA186_WR3_WR4.end',
        'J76 Blakelaw (Westbound)': 'CAJT_NCB6324_SR1_SR2.end.westbound'
    },
    'Traffic volumes on Jesmond Road and Coast Road': {
        'Central Motorway (Westbound)': 'CAJT_NCA1058_SR3_JR1.end.westbound',
        'Central Motorway (Eastbound)': 'CAJT_NCA1058_JR1_SR3.start.eastbound'
    },
    'Traffic volumes on River Tyne crossings': {
        'Tyne Bridge (Northbound)': 'CAJT_GHA167_DR1_TB.end.northbound',
        'Tyne Bridge (Southbound)': 'CAJT_GHA167_TB_DR1.start.southbound',
        # No plate read data for these, unfortunately.
        #'Redheugh Bridge (Northbound)': 'CAJT_GHA184_DWR1_RB.end.eastbound',
        #'Redheugh Bridge (Southbound)': 'CAJT_GHA184_RB_A1.start.westbound',
        #'Scotswood Bridge (Eastbound)': 'CAJT_NCA695_SB_SWR3.start.eastbound',
        #'Scotswood Bridge (Westbound)': 'CAJT_NCA695_SWR3_SB.end.westbound'
    },
    'Traffic volumes on Great North Road': {
        'Broadway (Southbound)': 'CAJT_NCB1318_GNR3_GNR2.start.southbound',
        'Broadway (Northbound)': 'CAJT_NCB1318_GNR2_GNR3.end.northbound'
    },
    'Traffic volumes within Sunderland': {
        'Stockton Rd (Southbound)': 'CAJT_SLA1231_GT1_RR1.start.southbound',
        'Stockton Rd (Northbound)': 'CAJT_SLA1231_RR1_GT1.end.northbound'
    }
}

def classifyTime(t):
    hour = int(t.strftime('%H'))
    if hour < 7:
        return 'Night (19:00 - 07:00)'
    elif hour < 10:
        return 'Morning peak (07:00 - 10:00)'
    elif hour < 16:
        return 'Inter-peak (10:00 - 16:00)'
    elif hour < 19:
        return 'Evening peak (16:00 - 19:00)'
    else:
        return 'Night (19:00 - 07:00)'

periodDurations = {
    'Morning peak (07:00 - 10:00)': 3,   # 7 - 10
    'Inter-peak (10:00 - 16:00)': 6,     # 10 - 16
    'Evening peak (16:00 - 19:00)': 3,   # 16 - 19
    'Night (19:00 - 07:00)': 12
}

allPeriods = list(periodDurations.keys())

for sensorName in summaryPoints.keys():
    dateIndex = []
    directionIndex = []
    summaryData = []
    
    columnsRequired = summaryPoints[sensorName].values()
    dfSensor = pdTrafficCount[columnsRequired].copy()
    dfSensor.insert(0, 'Date', dfSensor.index.to_series().apply(lambda t: t.date()))
    dfSensor.insert(0, 'Day of week', dfSensor.index.to_series().apply(lambda t: t.strftime('%A')))
    dfSensor.insert(1, 'Time of day', dfSensor.index.to_series().apply(lambda t: t.strftime('%H:%M:%S')))
    dfSensor.insert(1, 'Period', dfSensor.index.to_series().apply(classifyTime))

    dfDailyPeriodTotals = dfSensor.groupby(['Date', 'Day of week', 'Period'], as_index=False).sum()
    dfAveragePeriodTotals = dfDailyPeriodTotals.groupby(['Day of week', 'Period'], as_index=False).median()
    
    for d in range(daysToInclude, -1, -1):
        sensorDate = dateToday - pd.Timedelta(days=d)
        
        if sensorDate.strftime('%d/%m/%Y') in daysToSkip:
            continue
        
        dateIndex.extend(np.repeat(sensorDate.strftime('%A %d %B'), len(summaryPoints[sensorName].keys())))

        directionColumns = dfSensor.select_dtypes(plottableTypes).columns
        directionIndex.extend(summaryPoints[sensorName].keys())
                         
        dfSensorOnDate = dfSensor.loc \
            [(sensorDate <= dfSensor.index) &
            (dfSensor.index < sensorDate + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]
        dfSensorLastWeek = dfSensor.loc \
            [(sensorDate - pd.Timedelta(days=7) <= dfSensor.index) &
            (dfSensor.index < sensorDate - pd.Timedelta(days=7) + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]
        dfSensorYesterday = dfSensor.loc \
            [(sensorDate - pd.Timedelta(days=1) <= dfSensor.index) &
            (dfSensor.index < sensorDate - pd.Timedelta(days=1) + pd.Timedelta(hours=24))] \
            .copy() \
            .groupby(['Period']) \
            .agg(['sum', 'count']) \
            [directionColumns]

        dfSensorAverageDayOfWeek = dfAveragePeriodTotals[dfAveragePeriodTotals['Day of week'] == sensorDate.strftime('%A')]

        for direction in summaryPoints[sensorName]:
            directionColumn = summaryPoints[sensorName][direction]
            summaryRow = []
            for period in allPeriods:
                periodStats = dfSensorOnDate[dfSensorOnDate.index == period][directionColumn]
                if not periodStats['sum'].empty:

                    # Convert to an hourly value
                    periodTotal = periodStats['sum'].values[0]
                    periodHourly = periodTotal / periodStats['count'].values[0] * (3600 / trafficCountInterval)

                    # Change on yesterday
                    yesterdayHourly = dfSensorYesterday[dfSensorYesterday.index == period][directionColumn]
                    yesterdayHourly = (yesterdayHourly['sum'].values[0] / yesterdayHourly['count'].values[0]) * (3600 / trafficCountInterval)
                    if (sensorDate - pd.Timedelta(days=1)).strftime('%d/%m/%Y') in daysToSkip:
                        yesterdayChange = None
                    else:
                        yesterdayChange = (periodHourly - yesterdayHourly) / yesterdayHourly

                    # Change on last week
                    lastWeekHourly = dfSensorLastWeek[dfSensorLastWeek.index == period][directionColumn]
                    lastWeekHourly = (lastWeekHourly['sum'].values[0] / lastWeekHourly['count'].values[0]) * (3600 / trafficCountInterval)
                    lastWeekChange = (periodHourly - lastWeekHourly) / lastWeekHourly

                    # Change on normal profile
                    profileHourly = dfSensorAverageDayOfWeek[dfSensorAverageDayOfWeek['Period'] == period][directionColumn]
                    profileHourly = profileHourly.values[0] / periodDurations[period]
                    profileChange = (periodHourly - profileHourly) / profileHourly

                    summaryRow.extend([
                        periodHourly, # Total
                        yesterdayChange, # Change on yesterday
                        lastWeekChange, # Change on last week
                        profileChange, # Change on average
                    ])
                else:
                    summaryRow.extend(np.repeat(0.0, 4))     
            summaryData.append(summaryRow)

    rowIndex = pd.MultiIndex.from_arrays([
            dateIndex,
            directionIndex
        ],
        names=['Date', 'Direction']
    )

    formattersSummary = {}
    colPeriods = []
    colStats = []
    for period in allPeriods:
        formattersSummary[(period, 'Hourly average flow')] = '{:,.0f}'
        formattersSummary[(period, 'Change from day before (%)')] = '{:+,.0%}'
        formattersSummary[(period, 'Change from week before (%)')] = '{:+,.0%}'
        formattersSummary[(period, 'Change from annual average (%)')] = '{:+,.0%}'
        colPeriods.extend(np.repeat(period, 4))
        colStats.extend([
            'Hourly average flow',
            'Change from day before (%)',
            'Change from week before (%)',
            'Change from annual average (%)'
        ])

    colIndex = pd.MultiIndex.from_arrays(
        [colPeriods, colStats],
        names=['Period', 'Statistic']
    )

    dfSummary = pd.DataFrame(summaryData, columns=colIndex, index=rowIndex)
    dfSummaryStyler = dfSummary.style \
        .format(formattersSummary) \
        .set_caption(sensorName) \
        .set_table_styles(
            [dict(selector="th",props=[('text-align', 'center')]),
                dict(selector="tr:nth-child(2) th.col_heading",
                     props=[('vertical-align', 'bottom'),
                            ('writing-mode', 'vertical-rl'),
                            ]),
             dict(selector="caption", props=[('font-weight', 'bold'), ('font-size', '120%')]),
             dict(selector="tr th:nth-child(2)", props=[('white-space', 'nowrap'), ('min-width', '200px')])
            ]
        )

    periodBarColours = {
        'Morning peak (07:00 - 10:00)': '#FFA07A50',
        'Inter-peak (10:00 - 16:00)': '#EE1F5F50',
        'Evening peak (16:00 - 19:00)': '#FFA07A50',
        'Night (19:00 - 07:00)': '#A0FF7A50'
    }

    for period in allPeriods:
        dfSummaryStyler.background_gradient(
            subset=[(period, 'Change from annual average (%)')],
            vmin=-1.0,
            vmax=1.0,
            cmap='PiYG'
        )
        dfSummaryStyler.bar(subset=[(period, 'Hourly average flow')], color=periodBarColours[period], vmin=0)

    display(HTML(dfSummaryStyler._repr_html_().replace('+nan%', 'N/A')))